In [65]:
from IPython import get_ipython

import numpy as np
from numpy import random, dtype
from array import array
import math
from math import floor

import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib widget
# %matplotlib widget

from os import path, mkdir
from tqdm import tqdm

import ROOT
from ROOT import TCanvas, TFile, TPaveText, TMath, TF1, TH1, TH1F, TH2D, TH2F, TH3F, TRandom, TPolyMarker3D, TGraphErrors, TAxis, TGaxis
from ROOT import gROOT, gBenchmark, gRandom, gInterpreter, gPad, gStyle, gDirectory
from ROOT import nullptr, kBlue, kRed

# import rootpy
%jsroot on

In [66]:
ROOT.gROOT.LoadMacro('./includes/rootlogon.C')

0


 Beginning new ROOT session with private TStyle 


In [67]:
canvasCnt = 0
canvases = []

def NewCanvas(s1 = 800, s2 = 500):
    
    global canvasCnt, canvases
    n = "c_" + str(canvasCnt)
    canvases.append(TCanvas(n, n, s1, s2))
    canvases[canvasCnt].Draw()
    canvasCnt = canvasCnt+1
    return canvasCnt-1, canvases[canvasCnt-1]

In [68]:
def FillGraph(gra: TGraphErrors, x, y, ex, ey):
    gra.AddPoint(x, y)
    n = gra.GetN()
    gra.SetPointError(n-1, ex, ey)
    return n

In [69]:
run = "run530181"
out = F"out/{run}/"
exten = ".pdf"
fin = TFile(F"../data/roottople_new/{run}_new.root")
tre = fin.Get("t")

try:
    mkdir(out)
except:
    pass

mtMcp = "0.5*(tim[2]+tim[3])"
dtMcp = "tim[2]-tim[3]"
mtSipm = "0.5*(tim[1]+tim[0])"
dtSipm = "tim[1]-tim[0]"
mcSipm = "0.5*(charge[1]+charge[0])"
asymSipm = "(charge[1]-charge[0])/(charge[1]+charge[0])"
cutAnd = " && "

fidlarge = np.array([5.7, 6.7, 4.7, 5.6])
fidlarge = [5.7, 6.7, 4.7, 5.6] if run == "run530167" else fidlarge
fidlarge = [5.9, 7.0, 4.7, 5.6] if run == "run530167" else fidlarge
fidlarge = [5.9, 7.0, 4.5, 5.5] if run == "run530181" else fidlarge
fidlarge = [5.9, 7.0, 4.5, 5.5] if run == "run530178" else fidlarge
fidLarge = F"cryPos[0]>{fidlarge[0]} && cryPos[0]<{fidlarge[1]} && cryPos[1]>{fidlarge[2]} && cryPos[1]<{fidlarge[3]}" 

In [70]:
hist2 = TH2D("hist2", "hist2", 1000, 0, 500, 1000, -1, 1)
tre.Draw("(tim[0]-tim[1])/2 : charge[0]/2+charge[1]/2 >> hist2", "", "")
hist2 = gPad.GetPrimitive("hist2")
NewCanvas()
hist2.Draw("zcol")

In [71]:
eMin = 50
eSteps = 6
eMax = 300
steps = np.linspace(eMin, eMax, eSteps+1)
stepsNo = steps.shape[0]

fout = TFile(out + "reso.root", "recreate")

gra = TGraphErrors("gra")
gra.GetXaxis().SetTitle("mean SiPM charge [pC]")
gra.GetYaxis().SetTitle("mean time cell resolution [ns]")
gra.SetTitle(run)

for i, thisEne in enumerate(steps):
    if(i+1 == stepsNo): break
    sliceName = F"slice_{i}"
    hist2.GetXaxis().SetRangeUser(steps[i], steps[i+1])
    slice = hist2.ProjectionY()
    fitf = TF1("fitf", "gaus", -1, 1)
    fitf.SetParameter(1, slice.GetBinCenter(slice.GetMaximumBin()))
    fitf.SetParameter(2, 0.2*slice.GetRMS())
    slice.Fit(fitf, "REMQ")
    charg = hist2.ProjectionX()
    FillGraph(gra, charg.GetMean(), fitf.GetParameter(2), charg.GetRMS()/math.sqrt(charg.Integral()), fitf.GetParError(2))
    fout.cd()
    slice.Write()
    charg.Write()
    slice.Delete()

Error in <TGraphErrors::TGraphErrors>: Cannot open file: gra, TGraphErrors is Zombie


In [72]:
gra.Write()

qFrom, qTo = 50, 300
scalenpe = 2.48#*2
npeHeigth = 10e-3

gra.SetMinimum(npeHeigth/3)
gra.GetXaxis().SetRangeUser(qFrom, qTo)

In [73]:

_, cc = NewCanvas()
gStyle.SetOptFit(11)
gStyle.SetFitFormat("g")

fitf = TF1("fitf", "sqrt([0]*[0]/(x*x) + [1]*[1])")
fitf.SetParNames("a [ns#bulletpC]", "b [ns]")
# fitf = TF1("fitf", "sqrt([0]*[0]/(x*x) + [1]*[1]/x + [2]*[2])")

r = gra.Fit(fitf)

gra.SetMarkerStyle(20)
gra.SetMarkerSize(.5)

fnpe = TF1("fnpe", F"x*{scalenpe}")
fnpe.SetRange(fnpe.Eval(qFrom+10), fnpe.Eval(qTo-10))
anpe = TGaxis(qFrom+10, npeHeigth, qTo-10, npeHeigth, "fnpe", 510, "-")
anpe.SetTitle("N_{pe}")
anpe.SetTitleOffset(-1.1)
anpe.SetLabelSize(0.03)
anpe.SetLineColor(kBlue)
anpe.SetLabelColor(kBlue)
anpe.SetTextColor(kBlue)
#anpeSetTitleSize(0.06)

gra.Draw("AP")
anpe.Draw("same")

cc.SaveAs(out + "reso_2.pdf")
cc.SaveAs(out + "reso_2.C")

 FCN=12.7689 FROM MIGRAD    STATUS=CONVERGED      84 CALLS          85 TOTAL
                     EDM=3.18085e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  a [ns#bulletpC]   3.68605e+00   3.00830e-02   3.11637e-05  -1.78820e-03
   2  b [ns]       1.41720e-02   2.49493e-04   2.58461e-07   1.18068e-01


Info in <TCanvas::Print>: pdf file out/run530181/reso_2.pdf has been created
Info in <TCanvas::SaveSource>: C++ Macro file: out/run530181/reso_2.C has been generated


In [74]:
_, ccc = NewCanvas()
gStyle.SetOptFit(11)

fitf = TF1("fitf", "sqrt([0]*[0]/(x*x) + [1]*[1]/x + [2]*[2])")
fitf.SetParNames("a [ns#bulletpC]", "b [ns#bulletpC^{0.5}]", "c [ns]")

r = gra.Fit(fitf)

gra.SetMarkerStyle(20)
gra.SetMarkerSize(.5)

gra.SetMinimum(npeHeigth/3)
gra.GetXaxis().SetRangeUser(qFrom, qTo)
fnpe = TF1("fnpe", F"x*{scalenpe}")
fnpe.SetRange(fnpe.Eval(qFrom+10), fnpe.Eval(qTo-10))
anpe = TGaxis(qFrom+10, npeHeigth, qTo-10, npeHeigth, "fnpe", 510, "-")
anpe.SetTitle("N_{pe}")
anpe.SetTitleOffset(-1.1)
anpe.SetLabelSize(0.03)
anpe.SetLineColor(kBlue)
anpe.SetLabelColor(kBlue)
anpe.SetTextColor(kBlue)
#anpeSetTitleSize(0.06)

gra.Draw("AP")
anpe.Draw("same")

ccc.SaveAs(out + "reso_3.pdf")
ccc.SaveAs(out + "reso_3.C")

 FCN=12.6819 FROM MIGRAD    STATUS=CONVERGED     196 CALLS         197 TOTAL
                     EDM=2.08916e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  a [ns#bulletpC]   3.64956e+00   1.29965e-01   3.13752e-05   3.61717e-02
   2  b [ns#bulletpC^{0.5}]   6.03274e-02   1.03952e-01   1.17741e-05   9.80297e-02
   3  c [ns]       1.37814e-02   1.38919e-03   2.64944e-07   4.09914e+00


Info in <TCanvas::Print>: pdf file out/run530181/reso_3.pdf has been created
Info in <TCanvas::SaveSource>: C++ Macro file: out/run530181/reso_3.C has been generated


In [75]:
fout.Close()